# Lensing Systems Positions Table

In [4]:
# make a table in LaTeX format for easy pasting

import h5py
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Plots.model_plot import ModelPlot

# define filters and system names
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

center_ra = 3.99999999999998
center_dec = 1.9984014443252818e-15

# table format:
# Lens System Name, RA_lens, Dec_lens, RA_im1, Dec_im1, Ra_im2, Dec_im2, Image Separation [arcsec]
for i, name in enumerate(names):
    filename = f"../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"

    with h5py.File(filename, "r") as f:
        kwargs = {}
        for key in f:
            kwargs[key] = f[key][()]

    ra_lens = kwargs['center_x'] - center_ra
    dec_lens = kwargs['center_y'] - center_dec

    ra_im1 = kwargs['image_position'][0][0] - center_ra
    dec_im1 = kwargs['image_position'][1][0] - center_dec

    ra_im2 = kwargs['image_position'][0][1] - center_ra
    dec_im2 = kwargs['image_position'][1][1] - center_dec

    image_sep = np.sqrt((kwargs['image_position'][0][0] - kwargs['image_position'][0][1])**2 + (kwargs['image_position'][1][0] - kwargs['image_position'][1][1])**2)

    # print the data in LaTeX format
    print(f"{name} & {ra_lens:.3f} & {dec_lens:.3f} & {ra_im1:.3f} & {dec_im1:.3f} & {ra_im2:.3f} & {dec_im2:.3f} & {image_sep:.3f} \\\\")

J0407-5006 & -0.545 & -0.679 & -0.703 & -0.936 & 0.190 & 0.537 & 1.722 \\
J0806+2006 & 0.189 & -0.157 & -0.738 & 0.343 & 0.581 & -0.342 & 1.487 \\
J1442+4055 & 0.235 & -0.265 & -1.066 & 0.129 & 0.992 & -0.428 & 2.132 \\
J1515+1511 & 0.554 & -0.218 & -0.845 & 0.638 & 0.821 & -0.479 & 2.006 \\
J1620+1203 & -0.726 & -0.252 & -1.065 & -0.652 & 1.042 & 1.208 & 2.810 \\
J2325-5229 & -0.205 & -0.364 & -0.693 & 1.294 & 0.706 & -1.156 & 2.821 \\


# Magnitude Table

In [ ]:
import h5py
import numpy as np

# define filters and system names
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

# LaTeX table header
print("\\begin{tabular}{lccc|ccc}")
print("\\hline")
print("\\multicolumn{4}{c|}{F475X Magnitude} & \\multicolumn{3}{c}{F814W Magnitude} \\\\")
print("\\hline")
print("Lens System & Lens & Image A & Image B & Lens & Image A & Image B \\\\")
print("\\hline")

# Table content
for name in names:
    filename_f814w = f"../cutout_data/{name}/F814W/{name}_F814W_final.hdf5"
    with h5py.File(filename_f814w, "r") as f:
        kwargs_f814w = {key: f[key][()] for key in f}

    filename_f475xw = f"../cutout_data/{name}/F475X/{name}_F475X_final.hdf5"
    with h5py.File(filename_f814w, "r") as f:
        kwargs_f475x = {key: f[key][()] for key in f}

    # Lens
    lens_flux = kwargs_f814w['lens_flux']
    lens_flux_sigma = kwargs_f814w['lens_flux_sigma']
    lens_mag = kwargs_f814w['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    lens_flux_f475x = kwargs_f475x['lens_flux']
    lens_flux_sigma_f475x = kwargs_f475x['lens_flux_sigma']
    lens_mag_f475x = kwargs_f475x['lens_mag']
    lens_mag_sigmas_f475x = mag_sigma(lens_flux_f475x, lens_flux_sigma_f475x)

    # Image A
    im1_flux = kwargs_f814w['image_fluxes'][0]
    im1_flux_sigma = kwargs_f814w['image_fluxes_sigma'][0]
    im1_mag = kwargs_f814w['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    im1_flux_f475x = kwargs_f475x['image_fluxes'][0]
    im1_flux_sigma_f475x = kwargs_f475x['image_fluxes_sigma'][0]
    im1_mag_f475x = kwargs_f475x['image_mags'][0]
    im1_mag_sigmas_f475x = mag_sigma(im1_flux_f475x, im1_flux_sigma_f475x)

    # Image B
    im2_flux = kwargs_f814w['image_fluxes'][1]
    im2_flux_sigma = kwargs_f814w['image_fluxes_sigma'][1]
    im2_mag = kwargs_f814w['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    im2_flux_f475x = kwargs_f475x['image_fluxes'][1]
    im2_flux_sigma_f475x = kwargs_f475x['image_fluxes_sigma'][1]
    im2_mag_f475x = kwargs_f475x['image_mags'][1]
    im2_mag_sigmas_f475x = mag_sigma(im2_flux_f475x, im2_flux_sigma_f475x)

    # Table row
    print(f"{name} & "
          f"${lens_mag_f475x:.3f}^{{+{lens_mag_sigmas_f475x[0]:.3f}}}_{{-{lens_mag_sigmas_f475x[1]:.3f}}}$ & "
          f"${im1_mag_f475x:.4f}^{{+{im1_mag_sigmas_f475x[0]:.4f}}}_{{-{im1_mag_sigmas_f475x[1]:.4f}}}$ & "
          f"${im2_mag_f475x:.4f}^{{+{im2_mag_sigmas_f475x[0]:.4f}}}_{{-{im2_mag_sigmas_f475x[1]:.4f}}}$ & "
          f"${lens_mag:.3f}^{{+{lens_mag_sigmas[0]:.3f}}}_{{-{lens_mag_sigmas[1]:.3f}}}$ & "
          f"${im1_mag:.4f}^{{+{im1_mag_sigmas[0]:.4f}}}_{{-{im1_mag_sigmas[1]:.4f}}}$ & "
          f"${im2_mag:.4f}^{{+{im2_mag_sigmas[0]:.4f}}}_{{-{im2_mag_sigmas[1]:.4f}}}$ \\\\")

print("\\hline")
print("\\end{tabular}")



\begin{tabular}{lccc|ccc}
\hline
\multicolumn{4}{c|}{Flux [$e^-/s$]} & \multicolumn{3}{c}{Magnitude} \\
\hline
Lens System & Lens & Image A & Image B & Lens & Image A & Image B \\
\hline
J0407-5006 & $83.4^{+0.4}_{-0.5}$ & $193.36^{+0.10}_{-0.09}$ & $692.38^{+0.06}_{-0.06}$ & $20.307^{+0.006}_{-0.006}$ & $19.3943^{+0.0005}_{-0.0005}$ & $18.0099^{+0.0001}_{-0.0001}$ \\
J0806+2006 & $121.3^{+0.6}_{-0.7}$ & $221.46^{+0.05}_{-0.04}$ & $104.21^{+0.05}_{-0.05}$ & $19.902^{+0.006}_{-0.006}$ & $19.2482^{+0.0002}_{-0.0002}$ & $20.0661^{+0.0005}_{-0.0005}$ \\
J1442+4055 & $266.4^{+0.5}_{-0.6}$ & $603.31^{+0.08}_{-0.07}$ & $280.94^{+0.08}_{-0.07}$ & $19.048^{+0.002}_{-0.003}$ & $18.1599^{+0.0001}_{-0.0001}$ & $18.9898^{+0.0003}_{-0.0003}$ \\
J1515+1511 & $31.7^{+0.3}_{-0.3}$ & $595.55^{+0.03}_{-0.05}$ & $410.75^{+0.08}_{-0.07}$ & $21.360^{+0.011}_{-0.011}$ & $18.1765^{+0.0001}_{-0.0001}$ & $18.5773^{+0.0002}_{-0.0002}$ \\
J1620+1203 & $292.8^{+0.3}_{-0.4}$ & $67.43^{+0.05}_{-0.05}$ & $268.32^{+0.

# Lens Light Parameter Table

In [2]:
import pickle
import numpy as np
from scipy.special import gamma
import h5py

names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = "F814W"
source_names = ['J0806+2006', 'J1442+4055', 'J1620+1203']

# LaTeX header
print("\\begin{table*}[htb]")
print("\\centering")
print("\\caption{Lens Light and Source Light Model Parameters}")
print("\\label{tab:lens_light_source_params}")
print("\\begin{tabular}{lccc|ccc}")
print("\\hline")
print("\\multicolumn{5}{c|}{\\textbf{Lens Light}} & \\multicolumn{3}{c}{\\textbf{Source Light}} \\\\")
print("\\hline")
print("Lens System & $R_{\\text{S\\'ersic}}$ [arcsec] & $n_{\\text{S\\'ersic}}$ & $q$ & $R_{S\\'ersic}$ & $n_{S\\'ersic}$ & $q_{\\text{source}}$ \\\\")
print("\\hline")

for name in names:
    filename = f"../cutout_data/{name}/{filter}/{name}_{filter}_fit.pkl"
    hdf_file = f"../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"

    with open(filename, "rb") as f:
        loaded_data = pickle.load(f)

    with h5py.File(hdf_file, "r") as f:
        kwargs = {key: f[key][()] for key in f}

    chain_list = loaded_data.get('chain_list')

    sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list[1]

    # Get indices for the parameters we are interested in
    param_names = ['R_sersic_lens_light0', 'n_sersic_lens_light0', 'e1_lens_light0', 'e2_lens_light0',
                   'R_sersic_source_light0',  'n_sersic_source_light0', 'e1_source_light0', 'e2_source_light0']
    indices = [param_mcmc.index(p) for p in param_names]

    # Extract samples
    r_sersic_chain = samples_mcmc[:, indices[0]]
    n_sersic_chain = samples_mcmc[:, indices[1]]
    e1_chain = samples_mcmc[:, indices[2]]
    e2_chain = samples_mcmc[:, indices[3]]
    
    # Source light parameters
    rs_chain = samples_mcmc[:, indices[4]]
    ns_chain = samples_mcmc[:, indices[5]]
    e1_source_chain = samples_mcmc[:, indices[6]]
    e2_source_chain = samples_mcmc[:, indices[7]]

    # Compute axis ratio chains
    e_chain = np.sqrt(e1_chain**2 + e2_chain**2)
    q_chain = (1 - e_chain) / (1 + e_chain)

    e_source_chain = np.sqrt(e1_source_chain**2 + e2_source_chain**2)
    q_source_chain = (1 - e_source_chain) / (1 + e_source_chain)

    # Compute medians and uncertainties
    def get_median_and_uncertainties(samples):
        median = np.percentile(samples, 50)
        lower = median - np.percentile(samples, 16)
        upper = np.percentile(samples, 84) - median
        return median, lower, upper

    r_median, r_lower, r_upper = get_median_and_uncertainties(r_sersic_chain)
    n_median, n_lower, n_upper = get_median_and_uncertainties(n_sersic_chain)
    q_median, q_lower, q_upper = get_median_and_uncertainties(q_chain)
    
    if name in source_names:
        rs_median, rs_lower, rs_upper = get_median_and_uncertainties(rs_chain)
        ns_median, ns_lower, ns_upper = get_median_and_uncertainties(ns_chain)
        q_source_median, q_source_lower, q_source_upper = get_median_and_uncertainties(q_source_chain)

        print(f"{name} & "
              f"${r_median:.2f}^{{+{r_upper:.2f}}}_{{-{r_lower:.2f}}}$ & "
              f"${n_median:.2f}^{{+{n_upper:.2f}}}_{{-{n_lower:.2f}}}$ & "
              f"${q_median:.3f}^{{+{q_upper:.3f}}}_{{-{q_lower:.3f}}}$ & "
              f"${rs_median:.2f}^{{+{rs_upper:.2f}}}_{{-{rs_lower:.2f}}}$ & "
              f"${ns_median:.2f}^{{+{ns_upper:.2f}}}_{{-{ns_lower:.2f}}}$ & "
              f"${q_source_median:.3f}^{{+{q_source_upper:.3f}}}_{{-{q_source_lower:.3f}}}$ \\\\")
    else:
        print(f"{name} & "
              f"${r_median:.2f}^{{+{r_upper:.2f}}}_{{-{r_lower:.2f}}}$ & "
              f"${n_median:.2f}^{{+{n_upper:.2f}}}_{{-{n_lower:.2f}}}$ & "
              f"${q_median:.3f}^{{+{q_upper:.3f}}}_{{-{q_lower:.3f}}}$ & "
              f"$...$ & "
              f"$...$ & "
              f"$...$ \\\\")

print("\\hline")
print("\\end{tabular}")
print("\\end{table*}")


\begin{table*}[htb]
\centering
\caption{Lens Light and Source Light Model Parameters}
\label{tab:lens_light_source_params}
\begin{tabular}{lccc|ccc}
\hline
\multicolumn{5}{c|}{\textbf{Lens Light}} & \multicolumn{3}{c}{\textbf{Source Light}} \\
\hline
Lens System & $R_{\text{S\'ersic}}$ [arcsec] & $n_{\text{S\'ersic}}$ & $q$ & $R_{S\'ersic}$ & $n_{S\'ersic}$ & $q_{\text{source}}$ \\
\hline
J0407-5006 & $0.45^{+0.01}_{-0.01}$ & $4.95^{+0.01}_{-0.01}$ & $0.726^{+0.003}_{-0.004}$ & $...$ & $...$ & $...$ \\
J0806+2006 & $0.51^{+0.01}_{-0.01}$ & $4.05^{+0.02}_{-0.03}$ & $0.901^{+0.007}_{-0.007}$ & $0.04^{+0.01}_{-0.01}$ & $0.54^{+0.02}_{-0.02}$ & $0.178^{+0.002}_{-0.002}$ \\
J1442+4055 & $0.71^{+0.01}_{-0.01}$ & $4.95^{+0.01}_{-0.01}$ & $0.816^{+0.005}_{-0.004}$ & $0.21^{+0.01}_{-0.01}$ & $0.54^{+0.01}_{-0.01}$ & $0.361^{+0.003}_{-0.002}$ \\
J1515+1511 & $0.56^{+0.02}_{-0.01}$ & $1.71^{+0.02}_{-0.02}$ & $0.228^{+0.002}_{-0.002}$ & $...$ & $...$ & $...$ \\
J1620+1203 & $1.73^{+0.01}_{-0.01}$ 

In [10]:
import h5py
import numpy as np

# define filters and system names
names = ['J0407-5006', 'J0806+2006', 'J1442+4055', 'J1515+1511', 'J1620+1203', 'J2325-5229']
filter = 'F814W'

def mag_sigma(flux, sigma_flux):
    sigma_flux = np.asarray(sigma_flux)
    sigma_upper = sigma_flux[0]
    sigma_lower = sigma_flux[1]
    mag_err_upper = (2.5 / np.log(10)) * (sigma_upper / flux)
    mag_err_lower = (2.5 / np.log(10)) * (sigma_lower / flux)
    return mag_err_upper, mag_err_lower

print('FB / FA:')

# Table content
for name in names:
    filename = f"../cutout_data/{name}/{filter}/{name}_{filter}_final.hdf5"
    with h5py.File(filename, "r") as f:
        kwargs = {key: f[key][()] for key in f}

    # Lens
    lens_flux = kwargs['lens_flux']
    lens_flux_sigma = kwargs['lens_flux_sigma']
    lens_mag = kwargs['lens_mag']
    lens_mag_sigmas = mag_sigma(lens_flux, lens_flux_sigma)

    # Image A
    im1_flux = kwargs['image_fluxes'][0]
    im1_flux_sigma = kwargs['image_fluxes_sigma'][0]
    im1_mag = kwargs['image_mags'][0]
    im1_mag_sigmas = mag_sigma(im1_flux, im1_flux_sigma)

    # Image B
    im2_flux = kwargs['image_fluxes'][1]
    im2_flux_sigma = kwargs['image_fluxes_sigma'][1]
    im2_mag = kwargs['image_mags'][1]
    im2_mag_sigmas = mag_sigma(im2_flux, im2_flux_sigma)

    print(f'{name}: {im2_flux / im1_flux:.3f}')

FB / FA:
J0407-5006: 3.581
J0806+2006: 0.471
J1442+4055: 0.466
J1515+1511: 0.690
J1620+1203: 3.979
J2325-5229: 0.690
